In [18]:
%matplotlib inline
import numpy as np
import os

In [19]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.34.0


# Workspace 연결

In [20]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: mlrgeast
Azure region: eastus
Subscription id: 932c3e14-d0cf-4e41-9998-bdebb9bfa1cf
Resource group: mlrg


# 실험 연결

In [21]:
from azureml.core import Experiment
experiment_name = 'myExp'

exp = Experiment(workspace=ws, name=experiment_name)

### 생성할 모델 코드 확인

In [22]:
script_folder = './'
os.makedirs(script_folder, exist_ok=True)

with open(os.path.join(script_folder, 'myModel.py'), 'r') as f:
    print(f.read())

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



DATA_PATH="https://raw.githubusercontent.com/urtbest86/MLOps/master/result_train_dataset2.csv"
df = pd.read_csv(DATA_PATH)

DATA_PATH="https://raw.githubusercontent.com/urtbest86/MLOps/master/result_test_dataset2.csv"
test = pd.read_csv(DATA_PATH)

train=df.sample(frac=0.8)
val=df.sample(frac=0.2)

mean = train.mean(axis=0)
train -= mean
std = train.std(axis=0)
train /= std

test -= mean
test /= std

val-=mean
val/=std

train_data_set = train.values
x_train = train_data_set[:, 2:-1].astype(float)
y_train = train_data_set[:, -1].astype(float)

test_data_set = test.values
x_test = test_data_set[:, 2:-1].astype(float)
y_test = test_data_set[:, -1].astype(float)

val_data_set = val.values
x_val = val_data_set[:, 2:-1].astype(float)
y_val = val_data_set[:, -1].astype(float)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dens

# 기존 compute resource에 연결

In [6]:
'''from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "test-cluster")

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")'''

'from azureml.core.compute import AmlCompute\nfrom azureml.core.compute import ComputeTarget\nimport os\n\ncompute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "test-cluster")\n\nif compute_name in ws.compute_targets:\n    compute_target = ws.compute_targets[compute_name]\n    if compute_target and type(compute_target) is AmlCompute:\n        print("found compute target: " + compute_name)\nelse:\n    print("creating new compute target...")'

# compute resource 존재 확인 후 생성 또는 연결

In [23]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D11_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target: cpu-cluster


# Workspace에 환경 등록

In [24]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.6.2

- pip:
  - h5py<=2.10.0
  - azureml-defaults
  - tensorflow-gpu==2.0.0
  - keras<=2.3.1
  - matplotlib
  - pandas
  - scikit-learn

Overwriting conda_dependencies.yml


# 실험에 코드 실행 환경 등록

In [25]:
from azureml.core import Environment

env = Environment.from_conda_specification(name = 'keras-2.3.1', file_path = './conda_dependencies.yml')

'''# Specify a GPU base image
env.docker.enabled = True
env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.0-cudnn7-ubuntu18.04'''

env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210806.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "keras-2.3.1",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"
            ],
            "de

# 학습 실행 구성 및 제출
### ScriptRunConfig 만들기



모델에 대한 정보들 적어주기

In [26]:
from azureml.core import ScriptRunConfig

args = ['--data-folder', 
        '--batch-size', 4,
        '--first-layer-neurons', 1024,
        '--second-layer-neurons', 500,
        '--third-layer-neurons', 300,
        '--forth-layer-neurons', 200,
        '--learning-rate', 0.001]

src = ScriptRunConfig(source_directory=script_folder,
                      script='myModel.py',
                      arguments=args,
                      compute_target=compute_name,
                      environment=env)

In [31]:
run = exp.submit(src)

In [32]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [33]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
myExp,myExp_1635578180_f782f42a,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


실행 상태를 모니터링 하고 필요에 따라 실행 로그를 스트리밍

In [34]:
run.wait_for_completion(show_output=True)

RunId: myExp_1635578180_f782f42a
Web View: https://ml.azure.com/runs/myExp_1635578180_f782f42a?wsid=/subscriptions/932c3e14-d0cf-4e41-9998-bdebb9bfa1cf/resourcegroups/mlrg/workspaces/mlrgeast&tid=19411b3a-b7ca-48d9-a1e2-31694d0ccb65

Streaming azureml-logs/55_azureml-execution-tvmps_a7962f7b2ac9f1169c87ceb6cae64816af463a4730631f356922b7709eaa65dc_d.txt

2021-10-30T07:16:36Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/mlrgeast/azureml/myexp_1635578180_f782f42a/mounts/workspaceblobstore --tmp-path=/mnt/batch/tasks/shared/LS_root/jobs/mlrgeast/azureml/myexp_1635578180_f782f42a/caches/workspaceblobstore -o ro --file-cache-timeout-in-seconds=1000000 --cache-size-mb=89273 -o nonempty -o allow_other --config-file=/mnt/batch/tasks/shared/LS_root/jobs/mlrgeast/azureml/myexp_1635578180_f782f42a/configs/workspaceblobstore.cfg --log-level=LOG_WARNING
2021-10-30T07:16:36Z Successfully mounted a/an Blobfuse File System at /mnt/batch/tasks/shared/LS_root

{'runId': 'myExp_1635578180_f782f42a',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-10-30T07:16:34.624459Z',
 'endTimeUtc': '2021-10-30T07:17:11.173537Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '5671372d-58d1-43c8-b425-5ed854c15325',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'myModel.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--data-folder',
   '--batch-size',
   '4',
   '--first-layer-neurons',
   '1024',
   '--second-layer-neurons',
   '500',
   '--third-layer-neurons',
   '300',
   '--forth-layer-neurons',
   '200',
   '--learning-rate',
   '0.001'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'cpu-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datac

In [35]:
print(run.get_metrics())

{'Final test loss(mse)': 0.36707951805808325, 'Final test loss(mae)': 0.5233289003372192, 'loss VS val_loss': 'aml://artifactId/ExperimentRun/dcid.myExp_1635578180_f782f42a/loss VS val_loss_1635578208.png'}


In [36]:
print(run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_a7962f7b2ac9f1169c87ceb6cae64816af463a4730631f356922b7709eaa65dc_d.txt', 'azureml-logs/65_job_prep-tvmps_a7962f7b2ac9f1169c87ceb6cae64816af463a4730631f356922b7709eaa65dc_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_a7962f7b2ac9f1169c87ceb6cae64816af463a4730631f356922b7709eaa65dc_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/94_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'loss VS val_loss_1635578208.png', 'outputs/Youjin_test_model.pkl']


# 생성된 모델 등록

In [ ]:
'''model = run.register_model(model_name='myModelYoujin2', model_path='.')'''

In [37]:
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model = run.register_model(model_name='Youjin-test', 
                           model_path='outputs/Youjin_test_model.pkl',
                           model_framework=Model.Framework.TENSORFLOW,
                           model_framework_version='2.0',
                           )

print(model.name, model.id, model.version, sep='\t')

Youjin-test	Youjin-test:1	1


In [ ]:
'''os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)'''